In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_parquet(r'/Users/daviddangol/Documents/monthly report/domestic airlines/data/query-results_query-with-param_hotel_airlines_intl_bq_param_airlines_only_10-15-2025_11-01-2025000000000000.parquet')

In [4]:
display(df.head(5))

,user_id,event_name,event_date,event_timestamp,event_params,is_active_user,medium,source,platform,version,install_source
0,7121494,domesticFlightDetail,20251019,1760846335358639,"[{'key': 'firebase_conversion', 'value': {'str...",None,(none),(direct),ANDROID,4.6.1.0,com.heytap.market
1,9179299,domesticFlightDetail,20251019,1760889297504877,"[{'key': 'ga_session_id', 'value': {'string_va...",None,(none),(direct),ANDROID,4.7.0.1,manual_install
2,5313528,domesticFlightDetail,20251019,1760865055223239,"[{'key': 'firebase_event_origin', 'value': {'s...",None,(none),(direct),ANDROID,4.7.0.1,com.miui.huanji
3,11509078,domesticFlightDetail,20251101,1762020607158318,"[{'key': 'firebase_screen_class', 'value': {'s...",None,None,None,ANDROID,4.7.0.1,com.transsnet.store
4,3859929,domesticFlightDetail,20251017,1760700789486589,"[{'key': 'firebase_screen_class', 'value': {'s...",None,(none),(direct),ANDROID,4.7.0.1,com.coloros.backuprestore


In [5]:
funnel_data = {
    "Stage": ["domesticFlightSearch", "domesticFlightDetail", "domesticFlightPassengerDetail", "domesticFlightConfirmation"],
    "Count": [
        len(df[df['event_name'] == 'domesticFlightSearch'])+len(df[df['event_name'] == 'domesticFlightDetail'])+len(df[df['event_name'] == 'domesticFlightConfirmation'])+len(df[df['event_name'] == 'domesticFlightPassengerDetail']),
        len(df[df['event_name'] == 'domesticFlightDetail'])+len(df[df['event_name'] == 'domesticFlightConfirmation'])+len(df[df['event_name'] == 'domesticFlightPassengerDetail']),
        len(df[df['event_name'] == 'domesticFlightPassengerDetail']) +len(df[df['event_name'] == 'domesticFlightConfirmation']),
        len(df[df['event_name'] == 'domesticFlightConfirmation']) ]}

In [6]:
import plotly.graph_objects as go

funnel_df = pd.DataFrame(funnel_data)

# Calculate drop-off percentages
dropoff_percentages = [0] # Drop-off from a hypothetical stage before the first is 0
for i in range(1, len(funnel_df['Count'])):
    dropoff = funnel_df['Count'][i-1] - funnel_df['Count'][i]
    dropoff_percent = (dropoff / funnel_df['Count'][i-1]) * 100 if funnel_df['Count'][i-1] > 0 else 0
    dropoff_percentages.append(dropoff_percent)

# Create a list of text labels, including drop-off percentages
text_labels = []
for i, stage in enumerate(funnel_df['Stage']):
    conversion_rate = (funnel_df['Count'][i]/funnel_df['Count'][0]) * 100 if funnel_df['Count'][0] > 0 else 0
    if i == 0:
        text_labels.append(f"{funnel_df['Count'][i]} ({conversion_rate:.1f}%)")
    else:
         text_labels.append(f"{funnel_df['Count'][i]} ({conversion_rate:.1f}%) - {dropoff_percentages[i]:.1f}% Drop-off")


# Plot funnel chart using Plotly
fig = go.Figure(go.Funnel(
    y=funnel_df['Stage'],  # Y-axis represents the stages of the funnel
    x=funnel_df['Count'],  # X-axis represents the number of users at each stage
    textinfo="text",  # Show the custom text labels
    text=text_labels, # Assign the custom text labels
    textposition="auto" # Position the text outside the bars
))

# Update layout with a title and size
fig.update_layout(
    title="User Activation & Purchase Funnel with Drop-off Analysis",
    autosize=False, # Disable autosize
    width=1000, # Set the width of the chart
    height=900 # Set the height of the chart
)

# Show the interactive funnel chart
fig.show()